In [ ]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

using LinearAlgebra
using CairoMakie

# Background

The model is a pair of qubits. The qubits undergo ZZ crosstalk.

# Compute

In [ ]:
probs = Dict()

## Pair

In [ ]:
H_drift = zeros(4, 4)
# enter \otimes for kron
H_drives = [GATES[:X] ⊗ GATES[:I], GATES[:I] ⊗ GATES[:X]]
U_goal = GATES[:X] ⊗ GATES[:X]
ZZ = GATES[:Z] ⊗ GATES[:Z]
T = 50
Δt = 0.2

probs["pair"] = UnitarySmoothPulseProblem(
    H_drift, 
    H_drives, 
    U_goal, 
    T, 
    Δt;
    timesteps_all_equal=false,
    free_time=true,
    hessian_approximation=true
)

In [ ]:
solve!(probs["pair"]; max_iter=100)

In [ ]:
unitary_fidelity(probs["pair"])

Next function is not in the main code yet. It uses the robustness loss function.

In [ ]:
@views function infidelity_robustness(Hₑ::AbstractMatrix, prob::QuantumControlProblem)
    Z⃗ = vec(prob.trajectory.data)
    Z = prob.trajectory
    return InfidelityRobustnessObjective(Hₑ, Z).L(Z⃗, Z)
end

In [ ]:
infidelity_robustness(ZZ, probs["pair"])

## Robust pair

Introduce a new objective constructor for robustness.

```Julia
probs["robust"] = UnitaryRobustnessProblem(ZZ, probs["pair"])
```
would add a new ZZ robustness objective. It would do so while using the original problem constraints, the original problem objective, and enforcing the original problem fidelity as a constraint.

If we pass new constraints we can overwrite some of the original problem's features to get better solutions.

The next code is new, so message if there are any broken features.

In [ ]:
# Retaining original constraints and objective is too much for robustness
# reset the objective (see objective.jl)
objective = DefaultObjective()
objective += QuadraticRegularizer(:dda, probs["pair"].trajectory, 1e-4)
objective += QuadraticRegularizer(:a, probs["pair"].trajectory, 1e-2)

# Empty vector of type constraints (see constraints.jl)
constraints = AbstractConstraint[]

# Enforce a final fidelity constraint and optimize for robustness
probs["robust"] = UnitaryRobustnessProblem(
    ZZ, 
    probs["pair"];
    objective=objective,
    constraints=constraints,
    final_fidelity=0.9999, 
    verbose=false,
    hessian_approximation=true
)

In [ ]:
solve!(probs["robust"]; max_iter=150)

In [ ]:
# Fidelity constraint is satisfied
unitary_fidelity(probs["robust"])

In [ ]:
# Compare robustness
(x->println("Pair: $x"))(infidelity_robustness(ZZ, probs["pair"]))
(x->println("Robust: $x"))(infidelity_robustness(ZZ, probs["robust"]))

## Compare plots

In [ ]:
plot(probs["pair"].trajectory)

In [ ]:
plot(probs["robust"].trajectory)